In [1]:
import tabata as tbt
from tabata import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

def clean_data(input_file, output_file):
    ds = Opset(input_file)
    print(f"Taille du df sale : {len(ds)}")
    
    clean_ds = Opset("output_file")
    clean_ds.clean()
    
    for idx, df in enumerate(ds.iterator()):
        if max(df["F[N]"]) > 0:
            x = df.index
            t = (x - x[0]).total_seconds()
            dt = np.diff(t)
            i = np.argwhere(dt != dt[1])
            if len(i) > 0:
                name = df.index.name
                df.index = pd.date_range(x[0], periods=len(df), freq=x[1] - x[0])
                df.index.name = name 

            clean_ds.put(df)

    print(f"Taille du df après nettoyage initial : {len(clean_ds)}")
    
    final_clean_ds = Opset(output_file + "_final")
    final_clean_ds.clean()

    for idx, df in enumerate(clean_ds.iterator()):
        if "ALT[m]" in df.columns:
            altitude = df["ALT[m]"]

            inversed_altitude = -altitude
            minima_indices, _ = find_peaks(inversed_altitude, prominence=100)
            minima_values = altitude.iloc[minima_indices]

            has_fake_landing = False
            for idx_min in minima_indices:
                if idx_min != 0 and idx_min != len(altitude) - 1:
                    if altitude.iloc[idx_min] < 500:  
                        subsequent_altitudes = altitude.iloc[idx_min:]
                        if (subsequent_altitudes > altitude.iloc[idx_min] + 1000).any():
                            has_fake_landing = True
                            break
            if has_fake_landing:
                print(f"Vol exclu (faux atterrissage détecté) : Vol #{idx}")
                continue

        final_clean_ds.put(df)

    print(f"Taille du df final : {len(final_clean_ds)}")
    print(f"Fichier nettoyé et final sauvegardé sous {output_file}_final")

# Exemple d'appel
input_file = "../tabata/notebooks/data/in/AFL1EB.h5"
output_file = "../Data/cleaned/AFL1EB_cleaned.h5"
clean_data(input_file, output_file)


Taille du df sale : 52
Taille du df après nettoyage initial : 48
Vol exclu (faux atterrissage détecté) : Vol #6
Taille du df final : 47
Fichier nettoyé et final sauvegardé sous ../Data/cleaned/AFL1EB_cleaned.h5_final


In [16]:
Opset(output_file).plot()